# CUSTOM LOSS & METRICS FUNCTIONS

In [1]:
# Import Depedencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import import_ipynb
import Utils

2025-01-25 17:32:06.798599: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-25 17:32:06.798626: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-25 17:32:06.798629: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-25 17:32:06.798642: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-25 17:32:06.798653: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

In [2]:
# Dataset Batching
BATCH_SIZE=32
BUFFER_SIZE = 8
trainDataset = Utils.trainDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valDataset = Utils.valDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
testDataset = Utils.testDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

## CUSTOM BCE LOSS FUNCTION

In [3]:
# Custom Binary Cross-Entropy Loss
def customBCE(factor):
    def loss(yTrue, yPred):
        bce = tf.keras.losses.BinaryCrossentropy()
        return bce(yTrue, yPred)*factor
    return loss

In [4]:
# Model Compilation
Utils.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=customBCE(0.5)
)

In [5]:
# Training
Utils.model.fit(trainDataset, validation_data=valDataset, epochs=3, verbose=1)

Epoch 1/3


2025-01-25 17:32:07.611537: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


689/689 ━━━━━━━━━━━━━━━━━━━━ 42s 58ms/step - loss: 0.3207 - val_loss: 0.3425
Epoch 2/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 57ms/step - loss: 0.2472 - val_loss: 0.6033
Epoch 3/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - loss: 0.2723 - val_loss: 0.3108


## CUSTOM LOSS USING CLASS

In [6]:
# Loss Class
class CustomBCE(tf.keras.losses.Loss):
    def __init__(self, FACTOR=1):
        super(CustomBCE, self).__init__()
        self.FACTOR = FACTOR
    def call(self, yTrue, yPred):
        bce = tf.keras.losses.BinaryCrossentropy()
        return bce(yTrue, yPred) * self.FACTOR

In [7]:
# Model Compilation
Utils.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=CustomBCE(FACTOR=0.5)
)

In [8]:
# Training
Utils.model.fit(trainDataset, validation_data=valDataset, epochs=3, verbose=1)

Epoch 1/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 43s 59ms/step - loss: 0.2525 - val_loss: 0.2890
Epoch 2/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 56ms/step - loss: 0.1903 - val_loss: 0.2095
Epoch 3/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 57ms/step - loss: 0.1689 - val_loss: 0.1586


## CUSTOM ACCURACY METRICS

In [9]:
# Custom Binary Accuracy Function
def customAccuracy(FACTOR=1):
    def accuracy(yTrue, yPred):
        return tf.keras.metrics.binary_accuracy(yTrue, yPred) * FACTOR
    return accuracy

In [10]:
# Model Compilation
Utils.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=CustomBCE(FACTOR=0.5),
    metrics=[customAccuracy(FACTOR=0.9)]
)

In [11]:
# Training
Utils.model.fit(trainDataset, validation_data=valDataset, epochs=3, verbose=1)

Epoch 1/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 43s 59ms/step - accuracy: 0.8060 - loss: 0.1618 - val_accuracy: 0.8138 - val_loss: 0.1601
Epoch 2/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 40s 58ms/step - accuracy: 0.8108 - loss: 0.1576 - val_accuracy: 0.8095 - val_loss: 0.2003
Epoch 3/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.8110 - loss: 0.1496 - val_accuracy: 0.8141 - val_loss: 0.1918


## CUSTOM METRIC CLASS

In [12]:
# Class Metric
class CustomAccuracy(tf.keras.metrics.Metric):
  def __init__(self, name = 'Custom_Accuracy', FACTOR = 1):
    super(CustomAccuracy, self).__init__()
    self.FACTOR = FACTOR
    self.accuracy = self.add_weight(name = name, initializer = 'zeros')


  def update_state(self, y_true, y_pred, sample_weight = None):
    output = tf.keras.metrics.binary_accuracy(tf.cast(y_true, dtype = tf.float32), y_pred)*self.FACTOR
    self.accuracy.assign_add(tf.math.count_nonzero(output, dtype = tf.float32)/tf.cast(output.shape[1], dtype = tf.float32))

  def result(self):
    return self.accuracy

  def reset_states(self):
    self.accuracy.assign_add(0.)

In [13]:
# Model Compilation
Utils.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=CustomBCE(FACTOR=0.5),
    metrics=[CustomAccuracy(FACTOR=0.9)]
)

In [14]:
# Training
Utils.model.fit(trainDataset, validation_data=valDataset, epochs=3, verbose=1)

Epoch 1/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 43s 60ms/step - custom_accuracy: 9904.5127 - loss: 0.1622 - val_custom_accuracy: 2476.0000 - val_loss: 0.2325
Epoch 2/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 41s 59ms/step - custom_accuracy: 9953.9961 - loss: 0.1569 - val_custom_accuracy: 2514.0000 - val_loss: 0.1869
Epoch 3/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 44s 64ms/step - custom_accuracy: 10011.4639 - loss: 0.1462 - val_custom_accuracy: 2545.0000 - val_loss: 0.1334
